# Jupyter notebook based on ImageD11 to process scanning 3DXRD data
# Written by Haixing Fang, Jon Wright and James Ball
## Date: 16/01/2024

In [ ]:
import os

os.environ['OMP_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())
PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

In [ ]:
# import functions we need

import os, glob, pprint
import numpy as np
import h5py
from tqdm.notebook import tqdm

import matplotlib
%matplotlib widget
from matplotlib import pyplot as plt

from ImageD11.nbGui import nb_utils as utils

import ImageD11.grain
import ImageD11.indexing
import ImageD11.columnfile
from ImageD11.sinograms import properties, dataset

from ImageD11.blobcorrector import eiger_spatial

In [ ]:
# desination of H5 files
# replace below with e.g.:
# dset_path = '/data/visitor/expt1234/20240101/PROCESSED_DATA/sample/dataset/sample_dataset.h5'

dset_path = ''

In [ ]:
# load the dataset from file

ds = ImageD11.sinograms.dataset.load(dset_path)

print(ds)
print(ds.shape)

In [ ]:
ds.phases = ds.get_phases_from_disk()
ds.phases.unitcells

In [ ]:
# now let's select a phase to index from our parameters json
phase_str = 'Fe'

ucell = ds.phases.unitcells[phase_str]

print(ucell.lattice_parameters, ucell.spacegroup)

In [ ]:
# load 3d columnfile from disk

cf_3d = ds.get_cf_3d_from_disk()
ds.update_colfile_pars(cf_3d, phase_name=phase_str) 

In [ ]:
# load 2d columnfile from disk

cf_2d = ds.get_cf_2d_from_disk()
ds.update_colfile_pars(cf_2d, phase_name=phase_str) 

In [ ]:
# load grains from disk

grains = ds.get_grains_from_disk(phase_str)

In [ ]:
# Work out what 2D and 3D peaks we have for each grain
# These will be "masks" on cf_2d and cf_3d
# Will save a lot of speed

for inc, grain in enumerate(tqdm(grains)):
    grain.mask_3d = cf_3d.labels == inc
    grain.mask_2d = np.isin(cf_2d.spot3d_id, cf_3d.spot3d_id[grain.mask_3d])

In [ ]:
# let's say we want all the omega values for the 3D peaks of a grain
# all we have to do for this is get the omega values from cf_3d (all 3d peaks)
# you do this with cf_3d.omega
# then mask it by the mask_3d of that grain
# so you get cf_3d.omega[some_grain.mask_3d]

In [ ]:
# pick the grain with the most 3D peaks
selected_grain = sorted(grains, key=lambda g: sum(g.mask_3d), reverse=True)[0]

# and look at its 2D and 3D peaks

fig, ax = plt.subplots()
ax.scatter(cf_3d.fc[selected_grain.mask_3d], cf_3d.sc[selected_grain.mask_3d], marker="X", c=cf_3d.omega[selected_grain.mask_3d], label='Merged 3D peak')
cols = ax.scatter(cf_2d.fc[selected_grain.mask_2d], cf_2d.sc[selected_grain.mask_2d], c=cf_2d.omega[selected_grain.mask_2d], s=cf_2d.s_I[selected_grain.mask_2d] / 5000, label='Contibutory 2D peaks')
fig.colorbar(cols)
ax.set_xlim(0, 2048)
ax.set_ylim(0, 2048)
ax.invert_yaxis()
ax.legend()
ax.set_title("Color is omega of peak. Scaled by sum intensity")
ax.set_xlabel("f_raw")
ax.set_ylabel("s_raw")
ax.set_aspect(1)
plt.show()

In [ ]:
# take the selected grain
# get the omega value (corresponding to some image frame) with the most associated 2D peaks

unique, counts = np.unique(cf_2d.omega[selected_grain.mask_2d], return_counts=True)
hits_dict = dict(zip(unique, counts))
hits_dict_max = sorted(hits_dict.items(), key=lambda x: x[1], reverse=True)[0]
max_omega, peaks_on_frame = hits_dict_max

# mask all the omega values for the 2D peaks of the selected grain
# so we are only selecting those of max_omega

omega_mask = cf_2d.omega[selected_grain.mask_2d] == max_omega

# now extract the fast, slow and omega values of the peaks
# we are chaining masks together
# we are first selecting 2D peaks for the selected grain with [selected_grain.mask_2d]
# then we are selecting those with the correct omega value with [omega_mask]

fast, slow, omega = cf_2d.f_raw[selected_grain.mask_2d][omega_mask], cf_2d.s_raw[selected_grain.mask_2d][omega_mask], cf_2d.omega[selected_grain.mask_2d][omega_mask]

# print(fast, slow, omega)

# get the corresponding frame number

omegas_sorted = np.sort(ds.omega)[0]
omega_slop = np.round(np.diff(omegas_sorted).mean(), 3)

frame_number = np.where(np.isclose(ds.omega[0, :], max_omega, atol=omega_slop/10))[0][0]

# get the corresponding 2D frame
with h5py.File(ds.masterfile, 'r') as h5In:
    frame_2d = h5In['1.1/measurement/frelon3'][frame_number].astype('uint16')

In [ ]:
# plot it

from matplotlib.colors import LogNorm

fig, ax = plt.subplots()
ax.imshow(frame_2d, norm=LogNorm(vmax=1000), interpolation="nearest")
ax.scatter(fast, slow, marker='+', c="r")
ax.set_title(f"Frame {frame_number} at w = {max_omega:.2f}")
plt.show()

In [ ]:
# unfortunately due to the segmenter used, we have very little information about the 3D peaks
# we have the following columns in the columnfile:
# "s_raw": centre-of-mass of peak in slow direction
# "f_raw": centre-of-mass of peak in fast direction
# "omega": centre-of-mass of peak in omega
# "sum_intensity": sum of intensity of peak
# "Number_of_pixels": number of pixels in peak

# if you want 3D peak shape information, you need to calculate it!
# below is an example of a function (using Numba for speed) to do this
# it calculates the extent in slow, fast and omega for each 3D peak
# it uses the 2D peak information to do this

from numba import njit, prange

@njit(parallel=True)
def calculate_3d_peak_extents(index_3d, spot3d_id_2d, fc_2d, sc_2d, omega_2d):
    # make arrays to store the results in
    # same length as index_3d
    
    fast_extent = np.full(index_3d.shape, -1, dtype=fc_2d.dtype) # extent in fast direction
    slow_extent = np.full(index_3d.shape, -1, dtype=fc_2d.dtype)  # extent in slow direction
    omega_extent = np.full(index_3d.shape, -1, dtype=fc_2d.dtype)   # extent in omega direction
    
    # iterate through each of the 3D peaks
    for inc in prange(index_3d.shape[0]):
        # select 2D peaks that merged to form this 3D peak
        mask_2d = spot3d_id_2d == index_3d[inc]
        
        # get the fast, slow and omega for the 2D peaks
        peak_2d_fast = fc_2d[mask_2d]
        peak_2d_slow = sc_2d[mask_2d]
        peak_2d_omega = omega_2d[mask_2d]
        
        # use ptp (basically max-min) to determine extent
        fast_extent[inc] = np.ptp(peak_2d_fast)
        slow_extent[inc] = np.ptp(peak_2d_slow)
        omega_extent[inc] = np.ptp(peak_2d_omega)
        
    return fast_extent, slow_extent, omega_extent

In [ ]:
fast_extent, slow_extent, omega_extent = calculate_3d_peak_extents(cf_3d.spot3d_id, cf_2d.spot3d_id, cf_2d.fc, cf_2d.sc, cf_2d.omega)

In [ ]:
# store the results in the 3D peaks columnfile

cf_3d.addcolumn(fast_extent, "f_extent")  # extent in fast direction
cf_3d.addcolumn(slow_extent, "s_extent")  # extent in slow direction
cf_3d.addcolumn(omega_extent, "o_extent")  # extent in omega direction

In [ ]:
# validate that this function is working

# pick the first 3d peak with over 30 2d peaks
n_2d_peaks = -1
peak_index_3d = -1
while n_2d_peaks < 30:
    peak_index_3d += 1
    # get its 2D peaks
    peak_2d_mask = cf_2d.spot3d_id == cf_3d.spot3d_id[peak_index_3d]

    # make sure we have more than 1 2d peak
    n_2d_peaks = np.sum(peak_2d_mask)
    
print(f"3D peak {peak_index_3d} has {n_2d_peaks} 2D peaks")

# and get its extents
first_3d_peak_f_extent = cf_3d.f_extent[peak_index_3d]
first_3d_peak_s_extent = cf_3d.s_extent[peak_index_3d]
first_3d_peak_o_extent = cf_3d.o_extent[peak_index_3d]

# get the fast, slow and omega values
peak_2d_fast = cf_2d.fc[peak_2d_mask]
peak_2d_slow = cf_2d.sc[peak_2d_mask]
peak_2d_omega = cf_2d.omega[peak_2d_mask]

# work out extents and assert equality
assert np.max(peak_2d_fast) - np.min(peak_2d_fast) == first_3d_peak_f_extent
assert np.max(peak_2d_slow) - np.min(peak_2d_slow) == first_3d_peak_s_extent
assert np.max(peak_2d_omega) - np.min(peak_2d_omega) == first_3d_peak_o_extent

In [ ]:
# we could plot these results for a single grain

fig, ax = plt.subplots()
ax.hist(cf_3d.o_extent[selected_grain.mask_3d], bins=30)
ax.set_xlabel("Omega extent of 3D peaks")
ax.set_ylabel("Count")
ax.semilogy()
plt.show()

In [ ]:
# or we could compute the mean omega extent for each grain, and plot a scatter plot:

for grain in grains:
    grain.mean_omega_extent = np.mean(cf_3d.o_extent[grain.mask_3d])

centre_plot = False

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')
xx = [grain.translation[0] for grain in grains]
yy = [grain.translation[1] for grain in grains]
zz = [grain.translation[2] for grain in grains]
col = [grain.mean_omega_extent for grain in grains]
sizes = [0.01*(cf_3d.nrows) for grain in grains]
if centre_plot:
    scatterplot = ax.scatter(xx-np.mean(xx), yy-np.mean(yy), zz, c=col, s=sizes)
else:
    scatterplot = ax.scatter(xx, yy, zz, c=col, s=sizes)
ax.set_xlim(-200,200)
ax.set_ylim(-200,200)
ax.set_zlim(-200,200)
plt.colorbar(scatterplot)
ax.set_title("Grains coloured by omega extent")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
plt.show()

In [ ]:
# could plot a histogram too:

fig, ax = plt.subplots()
ax.hist([grain.mean_omega_extent for grain in grains], bins=30)
plt.show()

In [ ]:
# Guide to what the 2D peaks columns mean, from ImageD11/src/blobs.h:

# s_1 = 0, /* 1 Npix */
# s_I,     /* 2 Sum intensity */
# s_I2,    /* 3 Sum intensity^2 */
# s_fI,    /* 4 Sum f * intensity */
# s_ffI,   /* 5 Sum f * f* intensity */
# s_sI,    /* 6 Sum s * intensity */
# s_ssI,   /* 7 Sum s * s * intensity */
# s_sfI,   /* 8 Sum f * s * intensity */
# s_oI,    /* 9 sum omega * intensity */
# s_ooI,   /* 10 sum omega2 * intensity */
# s_soI,   /* 11 sum omega * s * intensity */
# s_foI,   /* 12 sum omega * f * intensity */

# mx_I,   /* 13 Max intensity */
# mx_I_f, /* 14 fast at Max intensity */
# mx_I_s, /* 15 slow at Max intensity */
# mx_I_o, /* 16 omega at max I */

# bb_mx_f, /* 17 max of f */
# bb_mx_s, /* 18 max of s */
# bb_mx_o, /* 19 max of omega */
# bb_mn_f, /* 20 min of f */
# bb_mn_s, /* 21 min of s */
# bb_mn_o, /* 22 min of o */

# avg_i, /* Average intensity */
# f_raw, /* fast centre of mass */
# s_raw, /* slow centre of mass */
# omega, /* omega centre of mass */
# m_ss,  /* moments */
# m_ff,
# m_oo,
# m_sf,
# m_so,
# m_fo,